In [2]:
############# IMPORTS ###############
from brian2 import *
from network import *
from input import *
from run import *
from analysis import *
from PIL import Image
import orjson
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys
import os
import pickle
from datetime import datetime


In [ ]:
##########GET INPUTS ############
def gen_brian_inputs():
    """
    Generates 3D Poisson input rates using Gabor filters for a set of images.
    This function performs the following steps:
    1. Defines parameters for Gabor filters including wavelengths, scaling factors, orientations, phase offsets, and aspect ratios.
    2. Creates Gabor filters using the specified parameters.
    3. Generates permutations of image identifiers.
    4. Loads and processes images from the specified path.
    5. Converts images to greyscale and normalizes them.
    6. Generates 3D Poisson input rates from the processed images using the Gabor filters.
    Returns:
        np.ndarray: A 3D array of Poisson input rates generated from the images.
    """

    import numpy as np

    lambdas = [0.8]  # Wavelengths
    betas = [1]  # Scaling factor for bandwidth
    thetas = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]  # Orientations
    psis = [0, np.pi]  # Phase offsets
    gammas = [0.5]  # Aspect ratio
    size = 6  # Gabor filter sizes
    gabor_filters = GaborFilters(size, lambdas, betas, thetas, psis, gammas)

    from itertools import product

    permutations = list(product(["c", "v"], repeat=3))
    image_path = "data/3N2P/"

    
    # Create an np array with 8 images and each size 124x124:
    images = np.zeros((8, 128, 128))
    image_no = 0
    print("analysing data")
    for t, l, r in permutations:
        print(f"t: {t}, l: {l}, r: {r}")
        image_paths = image_path + f"t{t}_l{l}_r{r}.jpg"
        print(image_paths)
        image = Image.open(image_paths)
        greyscale = image.convert("L")
        greyscale = np.array(greyscale) / 255.0
        images[image_no] = greyscale
        image_no += 1

    _3d_poisson_inputs = generate_3d_poisson_rates_from_filters(
        images,
        gabor_filters,
        neuron_size=64,
        image_size=128,
    )
    # This has the shape num_images, neuron_size, neuron_size, num_filters

    return _3d_poisson_inputs


In [ ]:
N_LAYERS = 4  # Number of layers to create
NO_EPOCHS = 60
STIMULUS_LENGTH = 100 * ms
NUM_INPUTS = 8
STORAGE = None  # It worked but should be falsy so be concerned - I think I check whether it's None not False
DESCRIBE_NETWORK = False
input_lambda_e = 30 * nS
TEST_STIMULUS_LENGTH = 250 * ms
RESTORE = (False,)
STORE = (False,)
RADII = {
    "efe": {1: 8, 2: 12, 3: 16},
    "ele": {1: 2, 2: 2, 3: 2, 4: 2},
    "ebe": {2: 8, 3: 8, 4: 8},
    "eli": {1: 2, 2: 2, 3: 2, 4: 2},
    "ile": {1: 4, 2: 4, 3: 4, 4: 4},
}
AVG_NO_CONNECTIONS = {
    "efe": {0: 50, 1: 100, 2: 100, 3: 100},
    "ele": {1: 10, 2: 10, 3: 10, 4: 10},
    "ebe": {1: 10, 2: 10, 3: 10, 4: 10},
    "eli": {1: 10, 2: 10, 3: 10, 4: 10},
    "ile": {1: 30, 2: 30, 3: 30, 4: 30},
}

In [ ]:
no_excitatory_neurons = 64*64
lambda_a = 6 * nsiemens
excitatory_model = """
            dv/dt = (V_rest-v)/tau_m + (ge*(V_reversal_e-v) + gi*(V_reversal_i-v) + ga*(V_reversal_a-v))/(tau_m*g_leak) : volt
            dge/dt = -ge/tau_ee : siemens
            dgi/dt = -gi/tau_ie : siemens
            dga/dt = -ga/tau_a : siemens
            Cm : farad  # Membrane capacitance
            g_leak : siemens  # Leak conductance
            V_reset: volt  # Reset potential
            V_rest : volt  # Resting potential
            V_reversal_a : volt  # Homeostatic potential
            V_reversal_e : volt  # Reversal potential for excitatory synapses
            V_reversal_i : volt  # Reversal potential for inhibitory synapses
            V_threshold: volt  # Threshold potential
            t_refract: second # Refractory period
            sigma : volt  # Noise term
            tau_m : second  # Membrane time constant
            tau_ee : second  # Time constant for excitatory-excitatory synapses
            tau_ie : second  # Time constant for inhibitory-excitatory synapses
            tau_a : second  # Time constant for homeostatic synapses
            """

e1 = NeuronGroup(
    N= no_excitatory_neurons,
    model=excitatory_model,
    threshold="v>V_threshold",
    reset="""v=V_reset
    ga+= lamda_a""",
    refractory="t_refract",
    method="rk4",
    name="e1",
)

e2 = NeuronGroup(
    N= no_excitatory_neurons,
    model=excitatory_model,
    threshold="v>V_threshold",
    reset="""v=V_reset
    ga+= lamda_a""",
    refractory="t_refract",
    method="rk4",
    name="e2",
)
e3 = NeuronGroup(
    N= no_excitatory_neurons,
    model=excitatory_model,
    threshold="v>V_threshold",
    reset="""v=V_reset
    ga+= lamda_a""",
    refractory="t_refract",
    method="rk4",
    name="e3",
)
e4 = NeuronGroup(
    N= no_excitatory_neurons,
    model=excitatory_model,
    threshold="v>V_threshold",
    reset="""v=V_reset
    ga+= lamda_a""",
    refractory="t_refract",
    method="rk4",
    name="e4",
)
excitatory_neuron_groups = [e1, e2, e3, e4]
for excitatory_group in excitatory_neuron_groups:
    excitatory_group.Cm = 500 * pF
    excitatory_group.g_leak = 25 * nS
    excitatory_group.V_threshold = -53 * mV
    excitatory_group.V_reset = -57 * mV
    excitatory_group.V_rest = -74 * mV
    excitatory_group.V_reversal_e = 0 * mV
    excitatory_group.V_reversal_i = -70 * mV
    excitatory_group.V_reversal_a = -90 * mV
    excitatory_group.t_refract = 2 * ms
    excitatory_group.tau_m = 20 * ms
    excitatory_group.tau_ee = 2 * ms
    excitatory_group.tau_ie = 5 * ms
    excitatory_group.tau_a = 80 * ms


TypeError: NeuronGroup.__init__() missing 2 required positional arguments: 'N' and 'model'

In [ ]:
no_inhibitory_neurons = 32*32
inhibitory_model = """
            dv/dt = (V_rest-v)/tau_m + (ge*(V_reversal_e-v))/(tau_m * g_leak) : volt
            dge/dt = -ge/tau_ei : siemens
            Cm : farad
            g_leak : siemens
            V_rest : volt
            V_reset: volt
            V_reversal_e : volt
            V_threshold: volt  # Threshold potential
            t_refract: second # Refractory period
            sigma : volt
            tau_m : second
            tau_ei : second
            """
i1 = NeuronGroup(
    N= no_inhibitory_neurons,
    model=inhibitory_model,
    threshold="v>V_threshold",
    reset="v=V_reset",
    refractory="t_refract",
    method="rk4",
    name="i1",
)
i2 = NeuronGroup(
    N= no_inhibitory_neurons,
    model=inhibitory_model,
    threshold="v>V_threshold",
    reset="v=V_reset",
    refractory="t_refract",
    method="rk4",
    name="i2",
)
i3 = NeuronGroup(
    N= no_inhibitory_neurons,
    model=inhibitory_model,
    threshold="v>V_threshold",
    reset="v=V_reset",
    refractory="t_refract",
    method="rk4",
    name="i3",
)
i4 = NeuronGroup(
    N= no_inhibitory_neurons,
    model=inhibitory_model,
    threshold="v>V_threshold",
    reset="v=V_reset",
    refractory="t_refract",
    method="rk4",
    name="i4",
)
inhibitory_neuron_groups = [i1, i2, i3, i4]
for inhibitory_group in inhibitory_neuron_groups:
    inhibitory_group.Cm = 214 * pF
    inhibitory_group.g_leak = 18 * nS
    inhibitory_group.V_threshold = -53 * mV
    inhibitory_group.V_reset = -58 * mV
    inhibitory_group.V_rest = -82 * mV
    inhibitory_group.V_reversal_e = 0 * mV
    inhibitory_group.t_refract = 2 * ms
    inhibitory_group.tau_m = 12 * ms
    inhibitory_group.tau_ei = 2 * ms

In [ ]:
stdp_model = """
            dapre/dt = -apre/tau_c : 1 (event-driven)
            dapost/dt = -apost/tau_d : 1 (event-driven)
            lambda_e: siemens
            alpha_C: 1
            alpha_D: 1
            tau_c: second
            tau_d: second
            w: 1
            plasticity: 1
            learning_rate: 1
            """
stdp_on_pre = """
            ge_post += lambda_e * w
            apre = apre + alpha_C
            w = clip(w - apost * learning_rate * plasticity, 0, 1)
            """
stdp_on_post = """
            apost = apost + alpha_D
            w = clip(w+ apre * plasticity * learning_rate, 0, 1)
            """

excit_non_stdp_model = """
        w: 1
        lambda_e: siemens
        """
excit_non_stdp_on_pre = """
        ge_post += lambda_e
        """
inhib_non_stdp_model = """
        w: 1
        lambda_i: siemens
        """
inhib_non_stdp_on_pre = """
        gi_post += lambda_i
        """


efe_1 = Synapses(e1, e2, model=stdp_model, on_pre=stdp_on_pre, on_post=stdp_on_post, name="efe_1")
ele_1 = Synapses(e1, e1, model=stdp_model, on_pre=stdp_on_pre, on_post=stdp_on_post, name="ele_1")
eli_1 = Synapses(e1, i1, model=excit_non_stdp_model, on_pre=excit_non_stdp_on_pre, name="eli_1")
ile_1 = Synapses(i1, e1, model=inhib_non_stdp_model, on_pre=inhib_non_stdp_on_pre, name="ile_1")

efe_2 = Synapses(e2, e3, model=stdp_model, on_pre=stdp_on_pre, on_post=stdp_on_post, name="efe_2")
ele_2 = Synapses(e2, e2, model=stdp_model, on_pre=stdp_on_pre, on_post=stdp_on_post, name="ele_2")
eli_2 = Synapses(e2, i2, model=excit_non_stdp_model, on_pre=excit_non_stdp_on_pre, name="eli_2")
ile_2 = Synapses(i2, e2, model=inhib_non_stdp_model, on_pre=inhib_non_stdp_on_pre, name="ile_2")
ebe_2 = Synapses(e2, e1, model=stdp_model, on_pre=stdp_on_pre, on_post=stdp_on_post, name="ebe_2")

efe_3 = Synapses(e3, e4, model=stdp_model, on_pre=stdp_on_pre, on_post=stdp_on_post, name="efe_3")
ele_3 = Synapses(e3, e3, model=stdp_model, on_pre=stdp_on_pre, on_post=stdp_on_post, name="ele_3")
eli_3 = Synapses(e3, i3, model=excit_non_stdp_model, on_pre=excit_non_stdp_on_pre, name="eli_3")
ile_3 = Synapses(i3, e3, model=inhib_non_stdp_model, on_pre=inhib_non_stdp_on_pre, name="ile_3")
ebe_3 = Synapses(e3, e2, model=stdp_model, on_pre=stdp_on_pre, on_post=stdp_on_post, name="ebe_3")

ele_4 = Synapses(e4, e4, model=stdp_model, on_pre=stdp_on_pre, on_post=stdp_on_post, name="ele_4")
eli_4 = Synapses(e4, i4, model=excit_non_stdp_model, on_pre=excit_non_stdp_on_pre, name="eli_4")
ile_4 = Synapses(i4, e4, model=inhib_non_stdp_model, on_pre=inhib_non_stdp_on_pre, name="ile_4")
ebe_4 = Synapses(e4, e3, model=stdp_model, on_pre=stdp_on_pre, on_post=stdp_on_post, name="ebe_4")

efes = [efe_1, efe_2, efe_3]
eles = [ele_1, ele_2, ele_3, ele_4]
elis = [eli_1, eli_2, eli_3, eli_4]
iles = [ile_1, ile_2, ile_3, ile_4]
ebes = [ebe_2, ebe_3, ebe_4]
synapses = efes + eles + elis + iles + ebes

In [ ]:
for synapse in synapses:
    synapse.learning_rate = 0.04

for efe in efes:
    efe.tau_c = 5 * ms
    efe.tau_d = 5 * ms
    efe.alpha_C = 0.5
    efe.alpha_D = 0.5
    efe.lambda_e = 30 * nS
    efe.plasticity = 1

for ele in eles:
    ele.tau_c = 5 * ms
    ele.tau_d = 5 * ms
    ele.alpha_C = 0.5
    ele.alpha_D = 0.5
    ele.lambda_e = 20 * nS
    efe.plasticity = 1

for eli in elis:
    eli.lambda_e = 20 * nS

for ile in iles:
    ile.lambda_i = 20 * nS

for ebe in ebes:
    ebe.tau_c = 5 * ms
    ebe.tau_d = 5 * ms
    ebe.alpha_C = 0.5
    ebe.alpha_D = 0.5
    ebe.lambda_e = 20 * nS
    efe.plasticity = 1



In [ ]:
RADII = {
    "efe": {1: 8, 2: 12, 3: 16},
    "ele": {1: 2, 2: 2, 3: 2, 4: 2},
    "ebe": {2: 8, 3: 8, 4: 8},
    "eli": {1: 2, 2: 2, 3: 2, 4: 2},
    "ile": {1: 4, 2: 4, 3: 4, 4: 4},
}
AVF_NO_CONNECTIONS = {
    "efe": {0: 50, 1: 100, 2: 100, 3: 100},
    "ele": {1: 10, 2: 10, 3: 10, 4: 10},
    "ebe": {1: 10, 2: 10, 3: 10, 4: 10},
    "eli": {1: 10, 2: 10, 3: 10, 4: 10},
    "ile": {1: 30, 2: 30, 3: 30, 4: 30},
}

efe_radii = [8, 12, 16]
ele_radii = [2, 2, 2, 2]
ebe_radii = [8, 8, 8]
eli_radii = [2, 2, 2, 2]
ile_radii = [4, 4, 4, 4]

efe_avg_no = [50, 100, 100, 100]
ele_avg_no = [10, 10, 10, 10]
ebe_avg_no = [10, 10, 10, 10]
eli_avg_no = [10, 10, 10, 10]
ile_avg_no = [30, 30, 30, 30]

def get_indexes_efe(radius):
    for post_index in range(no_excitatory_neurons):
        col = post_index % 64
        row = post_index // 64
        pre_indexes = []
        post_indexes = []
        for i in range(64):
            for j in range(64):
                if np.sqrt((i - row) ** 2 + (j - col) ** 2) <= radius:
                    pre_indexes.append(i * 64 + j)
        post_indexes = [post_index] * len(pre_indexes)
    return pre_indexes, post_indexes

def get_indexes_ele(radius):
    for post_index in range(no_excitatory_neurons):
        col = post_index % 64
        row = post_index // 64
        pre_indexes = []
        post_indexes = []
        for i in range(64):
            for j in range(64):
                if np.sqrt((i - row) ** 2 + (j - col) ** 2) <= radius:
                    pre_indexes.append(i * 64 + j)
        post_indexes = [post_index] * len(pre_indexes)
    return pre_indexes, post_indexes

def get_indexes_ebe(radius):
    for post_index in range(no_excitatory_neurons):
        col = post_index % 64
        row = post_index // 64
        pre_indexes = []
        post_indexes = []
        for i in range(64):
            for j in range(64):
                if np.sqrt((i - row) ** 2 + (j - col) ** 2) <= radius:
                    pre_indexes.append(i * 64 + j)
        post_indexes = [post_index] * len(pre_indexes)
    return pre_indexes, post_indexes

def get_indexes_eli(radius):
    for post_index in range(no_inhibitory_neurons):
        col = (post_index % 32 ) * 2
        row = (post_index // 32 ) * 2
        pre_indexes = []
        post_indexes = []
        for i in range(64):
            for j in range(64):
                if np.sqrt((i - row) ** 2 + (j - col) ** 2) <= radius:
                    pre_indexes.append(i * 64 + j)
        post_indexes = [post_index] * len(pre_indexes)
    return pre_indexes, post_indexes

def get_indexes_ile(radius):
    for post_index in range(no_excitatory_neurons):
        col = (post_index % 64) // 2
        row = (post_index // 64) // 2
        pre_indexes = []
        post_indexes = []
        for i in range(32):
            for j in range(32):
                if np.sqrt((i - row) ** 2 + (j - col) ** 2) <= radius:
                    pre_indexes.append(i * 32 + j)
        post_indexes = [post_index] * len(pre_indexes)
    return pre_indexes, post_indexes


